<a href="https://colab.research.google.com/github/GitMishka/1/blob/main/movein_clean.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from datetime import datetime, timedelta

# Load the data
file_path = 'movein2.csv'  # Adjust this to your file's path
df = pd.read_csv(file_path)

# Remove the first 10 rows of the DataFrame as the very first step
# df = df.iloc[10:].reset_index(drop=True)
# df.to_csv('test.csv')
# Process to label each row with its community
df['Community'] = None
current_community = None
for index, row in df.iterrows():
    if str(row['Unit']).startswith('Community:'):
        current_community = row['Unit'].split(':', 1)[1].strip()
    else:
        df.at[index, 'Community'] = current_community
df = df.dropna(subset=['Community'])  # Remove community header rows
df.reset_index(drop=True, inplace=True)

# Remove rows without a Move In Date
df = df.dropna(subset=['Move In\nDate'])

# Convert dates from string to datetime
df['Service Start\nDate'] = pd.to_datetime(df['Service Start\nDate'], errors='coerce')
df['Move In\nDate'] = pd.to_datetime(df['Move In\nDate'], errors='coerce')

# Calculate the previous month and year
today = datetime.now()
first_day_of_current_month = today.replace(day=1)
last_day_of_previous_month = first_day_of_current_month - timedelta(days=1)
previous_month = last_day_of_previous_month.month
previous_month_year = last_day_of_previous_month.year

# Filter out rows where Service Start Date is not in the previous month
df = df[(df['Service Start\nDate'].dt.month == previous_month) & (df['Service Start\nDate'].dt.year == previous_month_year)]

# Mapping communities to regions (add your mappings here)
community_to_region = {
    "The Lantern at Morning Pointe of Chattanooga (chtl)": "Appalachian",
    "Morning Pointe of Powell (powl)": "Cumberland",
    "Morning Pointe of Lexington (lexn)": "Bluegrass",
    "The Lantern at Morning Pointe of Lexington (lexl)": "Bluegrass",
    "Morning Pointe of Lexington-East (lexe)": "Bluegrass",
    "Morning Pointe of Chattanooga Shallowford (chtt)": "Appalachian",
    "Morning Pointe of Franklin TN (fktn)": "Cumberland",
    "Morning Pointe of Louisville (lvlm)": "Bluegrass",
    "Morning Pointe of Russell (russ)": "Bluegrass",
    "Morning Pointe of Richmond (rich)": "Bluegrass",
    "Morning Pointe of Hixson (hixn)": "Appalachian",
    "Morning Pointe of Athens (aths)": "Appalachian",
    "Morning Pointe of Brentwood (brwd)": "Cumberland",
    "Morning Pointe of Calhoun (calh)": "Appalachian",
    "Morning Pointe of Clinton (clin)": "Cumberland",
    "The Lantern at Morning Pointe of Collegedale (cgdl)": "Appalachian",
    "Morning Pointe of Columbia (colm)": "Cumberland",
    "Morning Pointe of Frankfort (frkt)": "Bluegrass",
    "The Lantern at Morning Pointe of Frankfort (frkl)": "Bluegrass",
    "Morning Pointe of Franklin (frln)": "Bluegrass",
    "Morning Pointe of Greenbriar (grnb)": "Appalachian",
    "Morning Pointe of Greeneville (grnv)": "Appalachian",
    "Morning Pointe of Lenoir City (lenc)": "Appalachian",
    "The Lantern at Morning Pointe of Lenoir City (lenl)": "Appalachian",
    "Morning Pointe of Tullahoma (tula)": "Cumberland",
    "Morning Pointe of Tuscaloosa (tusc)": "Cumberland",
    "Morning Pointe of Danville (danv)": "Bluegrass",
    "The Lantern at Morning Pointe of Russell (rusl)": "Bluegrass",
    "The Lantern at Morning Pointe of Louisville (lvll)": "Bluegrass",
    "Morning Pointe of Spring Hill (sprh)": "Cumberland",
    "The Lantern at Morning Pointe of Spring Hill (sprl)": "Cumberland",
    "The Lantern at Morning Pointe of Franklin TN (fktl)": "Cumberland",
    "Morning Pointe of Knoxville (knox)": "Appalachian",
    "The Lantern at Morning Pointe of Knoxville (knxl)": "Appalachian",
    "Morning Pointe of East Hamilton (eham)": "Appalachian",
    "Morning Pointe of Hardin Valley (hard)": "Appalachian",
    "The Lantern at Morning Pointe of Powell (pwll)": "Cumberland",
    "Morning Pointe of Happy Valley (hppy)": "Appalachian",
}


# Add the 'Region' column based on the 'Community' column
df['Region'] = df['Community'].map(community_to_region)

# Add 'Weights' column based on 'Privacy\nLevel'
df['Weights'] = df['Privacy\nLevel'].apply(lambda x: 1 if x == 'Single' else 0.5)

# Remove the first 10 rows of the DataFrame


# Create a timestamp for the filename
timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

# Save the cleaned and updated dataframe to a new CSV with timestamp in filename
filename = f'final_processed_{timestamp}.csv'
df.to_csv(filename, index=False)

# Display the filename for confirmation
print(f"File saved as: {filename}")

# Count the number of entries per region
weights_sum_by_region = df.groupby('Region')['Weights'].sum()
# Display the counts
print(weights_sum_by_region)


File saved as: final_processed_2024-03-15_17-55-26.csv
Region
Appalachian    49.5
Bluegrass      25.0
Cumberland     26.0
Name: Weights, dtype: float64


In [ ]:
weights_sum_by_region.to_csv(f'final_{timestamp}.csv')